In [ ]:
%load_ext autoreload

%autoreload 2

import pandas as pd

from pathlib import Path

from src.data_handling import (
    process_training_data,
    process_test_data,
    test_data_to_numeric_full,
    SparseNumericTestDataIO
)

data_path = Path() / "data"

# see https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
data = pd.read_csv(data_path / "imdb.csv")

test_train_ratio = 0.8
train_length = int(len(data)*0.8)
test_length = len(data)-train_length

print(f"{train_length=}")
print(f"{test_length=}")

In [ ]:
train_data_path = data_path / "train_context_counts.csv"
raw_train_data = data.iloc[:train_length]

compute = False
if compute:
    process_training_data(raw_train_data.review, raw_train_data.sentiment, train_data_path)
train_data = pd.read_csv(train_data_path)
print("Train data done")

train_data

In [ ]:


test_data_path = data_path / "test_data.csv"
raw_test_data = data.iloc[train_length:]
compute = False
if compute:
    process_test_data(
        raw_test_data.review, raw_test_data.sentiment, test_data_path
    )
print("Test data done")

test_data = pd.read_csv(test_data_path)
test_data

In [ ]:
numeric_test_data_path = data_path / "numeric_test_data.parquet"
compute = False
if compute:
    test_data_to_numeric_full(
        test_data.context,
        test_data.words,
        # probably don't want to take all words, just take some of the most usual
        train_data.word[:100],
        numeric_test_data_path,
    )
print("Numeric test data done")

with open(numeric_test_data_path, "rb") as f:
    numeric_test_data = pd.read_parquet(path=f)
numeric_test_data

In [ ]:

sparse_numeric_test_data_path = data_path / "sparse_numeric_test_data.dat"
compute = True
if compute:
    handler = SparseNumericTestDataIO(
        test_data.context,
        test_data.words,
        train_data.word[:5000],
    )
    handler.write(sparse_numeric_test_data_path)

sntd = SparseNumericTestDataIO.read(sparse_numeric_test_data_path)
sntd.indices_to_bool(sntd.context_and_indices[0][1])